In [ ]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
resized_m, resized_n = 30, 30

# Load images
def load_image(image_path, new_m, new_n):
  image = cv.imread(image_path)
  image = cv.resize(image, (new_m, new_n))
  cv2_imshow(image)
  return image

In [ ]:
def image_to_vector(image_matrix, n, m):
  image_vector = []
  for k in range(n): # columns
    for j in range(m): # rows
      image_vector.append(image_matrix[k][j][0])
  return image_vector

In [ ]:
def out_of_focus_psf(k, l, r):
  p = []
  for i in range(r):
    p_i = []
    for j in range(r):
      if (i-k)**2 + (j-l)**2 <= r**2:
        p_i.append(1/(math.pi * r**2))
      else:
        p_i.append(0)
    p.append(p_i)
  return p

In [ ]:
def gaussian_psf(k, l, s1, s2):
  p = []
  for i in range(s1):
    p_i = []
    for j in range(s2):
      entry = np.exp(-1/2*(((i-k)/s1)**2 + ((j-l)/s2)**2))
      p_i.append(entry)
    p.append(p_i)
  return p

In [ ]:
def psf_to_vector(psf, n, m):
  psf_vector = []
  for k in range(n): # columns
    for j in range(m): # rows
      psf_vector.append(psf[j][k])
  return psf_vector

In [ ]:
def construct_blurring_matrix(psf_vector, m, n):
    A = []
    l = len(psf_vector)
    initial_index = l // 2 + 1

    # For every row
    for i in range(n):
        if initial_index < l:
            len_p = len(psf_vector[:initial_index])
            row = list(reversed(psf_vector[:initial_index])) + [0 for j in range(n - len_p)]
            A.append(row)
            initial_index += 1

        else:
            shift = abs(l - initial_index)
            len_p = len(psf_vector[:initial_index])
            row = [0 for j in range(shift)] + list(reversed(psf_vector[:initial_index])) \
                  + [0 for j in range(n - len_p - shift)]
            A.append(row[:n])
            initial_index += 1

    return A

In [ ]:
def vector_to_image(vector):
  n = int(math.sqrt(len(vector)))
  image_matrix = []
  for i in range(n):
    row = []
    for j in range(i*n, (i+1)*n):
      entry = vector[j]
      row.append([entry for i in range(3)])
    image_matrix.append(row)
  return image_matrix

In [ ]:
def blur_image(path, psf_array):
  dot_image = load_image(path, resized_m, resized_n)
  x = image_to_vector(dot_image, resized_n, resized_m) # our original image
  
  blurring_matrix = construct_blurring_matrix(psf_array, resized_n**2, resized_m**2) # matrix A

  blurred_vector = [i for i in np.matmul(blurring_matrix, x)]
  blurred_image = vector_to_image(blurred_vector)
  return blurred_vector, blurred_image, blurring_matrix

In [ ]:
def write_image(image, new_image_path, resized_m, resized_n):
  converted_image = cv.UMat(np.array(image, dtype=np.uint64))
  resized = cv.resize(converted_image, (resized_n, resized_m))
  cv.imwrite(new_image_path, resized)

In [ ]:
def naive_deblur(blurring_image, blurred_vector):
  blurring_inverse = np.linalg.inv(blurring_image)
  x = np.matmul(blurring_inverse, blurred_vector) # x - original image, x = A^(-1)*b
  x_matrix = vector_to_image(x)
  return x_matrix

# out-of-focus
x_coordinate, y_coordinate, radius = 1, 1, 5
psf_matrix = out_of_focus_psf(x_coordinate, y_coordinate, radius)

# gaussian
# x_coordinate, y_coordinate, s1, s2 = 15, 15, 10, 10
# psf_matrix = gaussian_psf(x_coordinate, y_coordinate, s1, s2)

psf_array = psf_to_vector(psf_matrix, 5, 5) # our psf array

blurred_vector, blurred_image, blurring_matrix = blur_image('/root/dot_image.jpg', psf_array)
write_image(blurred_image, '/root/blurred_dot_image.jpg', resized_n, resized_m)

restored_dot_image = naive_deblur(blurring_matrix, blurred_vector)
write_image(restored_dot_image, '/root/restored_dot_image.jpg', resized_m, resized_n)

In [ ]:
def reconstruct(k, A, b):
  U, s, V = np.linalg.svd(A)
  A_inverse = np.matrix(U[:, :k]) * np.diag(1/s[:k]) * np.matrix(V[:k, :])
  return np.asarray(np.matmul(A_inverse, b))[0]
  


In [ ]:
def truncated_reconstruction(blurring_matrix, blurred_vector, k):
  xk = reconstruct(k, blurring_matrix, blurred_vector)
  truncated = vector_to_image(xk)
  return truncated

blurred_vector, blurred_daisy, blurring_matrix = blur_image('/root/daisy.jpg', psf_array)
write_image(blurred_daisy, '/root/blurred_daisy.jpg', resized_m, resized_n)

k = 600
truncated = truncated_reconstruction(blurring_matrix, blurred_vector, k)
write_image(truncated, '/root/truncated_restored_daisy.jpg', resized_m, resized_n)

naive_daisy_deblur = naive_deblur(blurring_matrix, blurred_vector)
write_image(naive_daisy_deblur, '/root/naive_restored_daisy.jpg', resized_m, resized_n)

In [ ]:
def noising_the_vector(vector_):
  n = len(vector_)
  e = np.random.normal(size=n)
  e = e/np.linalg.norm(e)
  vector_ = vector_ + 0.01*np.linalg.norm(vector_)*e
  return vector_


In [ ]:
def noise_the_image(vector_):
  noised_vector = noising_the_vector(vector_)
  noised_image = vector_to_image(noised_vector)
  return noised_image

In [ ]:
def denoise_the_image(blurring_matrix, blurred_vector, k):
  xk = reconstruct(k, blurring_matrix, blurred_vector)
  truncated = vector_to_image(xk)
  return truncated

noised = noise_the_image(blurred_vector)
write_image(noised, '/root/noised_daisy.jpg', resized_m, resized_n)

k = 600
truncated = denoise_the_image(blurring_matrix, blurred_vector, k)
write_image(truncated, '/root/denoised_daisy.jpg', resized_m, resized_n)
